In [92]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import joblib

df = pd.read_csv('cleaned_data.csv')
numerical_cols = df.select_dtypes(include=['number']).columns
# Get only the first row (after removing the first column)
example_data = df.head(1)
example_data = example_data.iloc[:,1:]
# Save to a new CSV file (with header)
example_data_2 = df.iloc[[57]]  # Select the 58th row as a DataFrame
example_data_2 = example_data_2.drop(df.columns[0], axis=1)
example_data.to_csv("example1.csv", index=False)
example_data_2.to_csv("example2.csv", index=False)
df
identifier = pd.read_csv("gene_symbol_convertor/illumina_identifier.csv")
identifier

,ID,Species,Source,Search_Key,Transcript,ILMN_Gene,Source_Reference_ID,RefSeq_ID,Entrez_Gene_ID,GI,...,Chromosome,Probe_Chr_Orientation,Probe_Coordinates,Cytoband,Definition,Ontology_Component,Ontology_Process,Ontology_Function,Synonyms,GB_ACC
0,ILMN_1722532,Homo sapiens,RefSeq,ILMN_25544,ILMN_25544,JMJD1A,NM_018433.3,NM_018433.3,55818,46358420,...,2,+,86572991-86573040,2p11.2e,Homo sapiens jumonji domain containing 1A (JMJ...,nucleus [goid 5634] [evidence IEA],chromatin modification [goid 16568] [evidence ...,oxidoreductase activity [goid 16491] [evidence...,JHMD2A; JMJD1; TSGA; KIAA0742; DKFZp686A24246;...,NM_018433.3
1,ILMN_1708805,Homo sapiens,RefSeq,ILMN_10519,ILMN_10519,NCOA3,NM_006534.2,NM_006534.2,8202,32307123,...,20,+,45718934-45718983,20q13.12c,Homo sapiens nuclear receptor coactivator 3 (N...,nucleus [goid 5634] [pmid 9267036] [evidence NAS],"positive regulation of transcription, DNA-depe...",acyltransferase activity [goid 8415] [evidence...,CAGH16; TNRC14; pCIP; ACTR; MGC141848; CTG26; ...,NM_006534.2
2,ILMN_1672526,Homo sapiens,RefSeq,ILMN_17234,ILMN_17234,LOC389834,NM_001013655.1,NM_001013655.1,389834,61966764,...,4,-,51062-51111,NaN,Homo sapiens hypothetical gene supported by AK...,NaN,NaN,NaN,NaN,NM_001013655.1
3,ILMN_1703284,Homo sapiens,RefSeq,ILMN_502,ILMN_502,SPIRE2,NM_032451.1,NM_032451.1,84501,55749599,...,16,+,88465064-88465113,16q24.3b,Homo sapiens spire homolog 2 (Drosophila) (SPI...,NaN,NaN,zinc ion binding [goid 8270] [evidence IEA],MGC117166; Spir-2,NM_032451.1
4,ILMN_2185604,Homo sapiens,RefSeq,ILMN_19244,ILMN_19244,C17ORF77,NM_152460.2,NM_152460.2,146723,48255961,...,17,+,70101790-70101839,17q25.1b,Homo sapiens chromosome 17 open reading frame ...,NaN,NaN,NaN,FLJ31882,NM_152460.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24521,ILMN_1756443,Homo sapiens,RefSeq,ILMN_10181,ILMN_10181,INHA,NM_002191.2,NM_002191.2,3623,9257223,...,2,+,220148531-220148580,2q35f,"Homo sapiens inhibin, alpha (INHA), mRNA.",extracellular region [goid 5576] [pmid 1245695...,skeletal development [goid 1501] [pmid 1086521...,cytokine activity [goid 5125] [pmid 12456957] ...,NaN,NM_002191.2
24522,ILMN_1678051,Homo sapiens,RefSeq,ILMN_137361,ILMN_137361,HOXA3,NM_153632.1,NM_153632.1,3200,24497514,...,7,-,27120091-27120140,7p15.2a,"Homo sapiens homeo box A3 (HOXA3), transcript ...",nucleus [goid 5634] [evidence IEA],"regulation of transcription, DNA-dependent [go...",transcription factor activity [goid 3700] [evi...,MGC10155; HOX1E; HOX1,NM_153632.1
24523,ILMN_1714567,Homo sapiens,RefSeq,ILMN_28759,ILMN_28759,AHNAK,NM_001620.1,NM_001620.1,79026,61743953,...,11,-,62040123-62040172,11q12.3a,"Homo sapiens AHNAK nucleoprotein (AHNAK), tran...",nucleus [goid 5634] [pmid 1608957] [evidence NAS],NaN,protein binding [goid 5515] [evidence IEA],MGC5395; AHNAKRS,NM_001620.1
24524,ILMN_1759757,Homo sapiens,RefSeq,ILMN_3130,ILMN_3130,IGSF22,NM_173588.1,NM_173588.1,283284,27734790,...,11,-,18682635-18682661:18683940-18683962,11p15.1c,"Homo sapiens immunoglobulin superfamily, membe...",membrane [goid 16020] [evidence IEA],cell adhesion [goid 7155] [evidence IEA],protein binding [goid 5515] [evidence IEA],FLJ37794,NM_173588.1


In [93]:
X = df.drop('RESULT',axis=1).copy()
X.head()

,ILMN_1343291,ILMN_1651209,ILMN_1651228,ILMN_1651229,ILMN_1651235,ILMN_1651236,ILMN_1651237,ILMN_1651238,ILMN_1651254,ILMN_1651260,...,ILMN_2415722,ILMN_2415748,ILMN_2415776,ILMN_2415786,ILMN_2415826,ILMN_2415898,ILMN_2415911,ILMN_2415926,ILMN_2415949,ILMN_2415979
0,0.225505,-0.033261,-0.208953,0.188354,0.118059,-0.027438,0.161106,-0.038316,0.183315,0.206364,...,-2.126677,-0.133830,0.123104,-0.160362,0.065602,0.045352,-0.065240,0.085547,-0.065264,-0.025963
1,0.734810,0.518430,-0.903632,0.369321,0.406270,0.600751,0.520101,0.715021,-0.158329,0.657478,...,-0.845021,-0.390985,0.574927,-1.174522,0.678281,-0.073195,-0.142896,-0.205349,-0.152270,-0.749829
2,0.743191,0.599034,-0.351196,0.393824,0.588420,0.635966,0.532986,0.563128,0.126089,0.589630,...,-1.719068,-0.525717,0.371021,-0.346104,0.662314,-0.389993,0.081490,0.011367,-0.179159,-0.491612
3,0.632620,0.251444,-0.061795,0.147674,0.466300,0.278022,0.447510,0.311911,0.929209,0.411162,...,-0.011921,0.641134,0.241069,0.417610,0.359925,-0.093669,-0.021467,0.049962,-0.216873,0.322534
4,-0.177734,-0.164714,1.253607,-0.214611,-0.387988,-0.435218,-0.251590,-0.420771,0.579907,-0.178503,...,-0.010732,0.525030,-0.421567,-0.631475,-0.041858,-0.243361,-0.274051,-0.718539,-0.118360,0.928707


In [94]:
y = df['RESULT'].copy()
y.head()

0    STROKE
1    STROKE
2    STROKE
3    STROKE
4    STROKE
Name: RESULT, dtype: object

In [95]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2)
X_train_scaled = scale(X_train)
X_test_scaled = scale(X_test)

In [96]:
m1 = svm.LinearSVC(C=1)
m1.fit(X_train,y_train)
m1.score(X_train,y_train)
joblib.dump(m1, 'svm.joblib')

['svm.joblib']

In [97]:
y_pred = m1.predict(X_test)
pd.crosstab(y_test,y_pred)

col_0,CONTROL,STROKE
RESULT,,
CONTROL,6,0
STROKE,0,13


In [98]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     CONTROL       1.00      1.00      1.00         6
      STROKE       1.00      1.00      1.00        13

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



In [99]:
importance = m1.coef_[0]

indices = np.argsort(np.abs(importance))[::-1]
feature_names = X_train.columns

# Get sorted feature names by importance
sorted_feature_names = feature_names[indices]
top_n = 10
for i in range(top_n):
    print(f"{i+1}. {sorted_feature_names[i]} - importance: {importance[indices[i]]:.4f}")

1. ILMN_1697499 - importance: 0.0240
2. ILMN_2165753 - importance: -0.0170
3. ILMN_1715169 - importance: 0.0134
4. ILMN_1661266 - importance: -0.0094
5. ILMN_1783142 - importance: -0.0093
6. ILMN_2116877 - importance: 0.0092
7. ILMN_1776519 - importance: 0.0081
8. ILMN_1812281 - importance: 0.0076
9. ILMN_1717163 - importance: 0.0066
10. ILMN_2150787 - importance: 0.0063


In [100]:
# For each top feature, find the matching row in the identifier CSV and print info
for i in range(top_n):
    probe_id = sorted_feature_names[i]
    imp = importance[indices[i]]

    # Search for the row where the identifier column matches the probe ID
    match_row = identifier[identifier['ID'] == probe_id]
    #print(match_row['Entrez_Gene_ID'])
    if not match_row.empty:
        enzyme_id = match_row['Entrez_Gene_ID'].values[0]  # replace with your actual column name
        print(f"{i+1}. {probe_id} - importance: {imp:.4f} - Entrez_GENE_ID: {enzyme_id}")
    else:
        print(f"{i+1}. {probe_id} - importance: {imp:.4f} - Entrez_GENE_ID: Not Found")

1. ILMN_1697499 - importance: 0.0240 - Entrez_GENE_ID: 3127
2. ILMN_2165753 - importance: -0.0170 - Entrez_GENE_ID: 649853
3. ILMN_1715169 - importance: 0.0134 - Entrez_GENE_ID: 3123
4. ILMN_1661266 - importance: -0.0094 - Entrez_GENE_ID: 3119
5. ILMN_1783142 - importance: -0.0093 - Entrez_GENE_ID: 6192
6. ILMN_2116877 - importance: 0.0092 - Entrez_GENE_ID: 10562
7. ILMN_1776519 - importance: 0.0081 - Entrez_GENE_ID: 5909
8. ILMN_1812281 - importance: 0.0076 - Entrez_GENE_ID: 383
9. ILMN_1717163 - importance: 0.0066 - Entrez_GENE_ID: 2162
10. ILMN_2150787 - importance: 0.0063 - Entrez_GENE_ID: 3107


In [ ]:
for i in range(10):
    probe_id = sorted_feature_names[i]
    imp = importance[indices[i]]

    # Search for the row where the identifier column matches the probe ID
    match_row = identifier[identifier['ID'] == probe_id]
    #print(match_row['Entrez_Gene_ID'])
    if not match_row.empty:
        enzyme_id = match_row['Entrez_Gene_ID'].values[0]  # replace with your actual column name
        print(enzyme_id)
    else:
        print(f"{i+1}. {probe_id} - importance: {imp:.4f} - Entrez_GENE_ID: Not Found")

3127
649853
3123
3119
6192
10562
5909
383
2162
3107


In [101]:
m1.coef_

array([[-2.44016548e-04, -1.11270342e-03, -9.13265952e-05, ...,
        -1.34585684e-03, -4.56341437e-04,  2.13445391e-03]])